In [240]:
import pandas as pd
import cupy as cp
import numpy as np
from tqdm import tqdm

In [241]:

bianca = [r'C:\Users\harry\Documents\b/m5-2_BW_20200715-115613_scored_behaviour.csv',r'C:\Users\harry\Documents\b/f10-2_BW_20200715-162501_scored_behaviour.csv',r'C:\Users\harry\Documents\b/f6-4_BW_20200715-192131_scored_behaviour.csv']

rhianne =[r'C:\Users\harry\Documents\r/M5.2_20200715-101855_scored_behaviour.csv',r'C:\Users\harry\Documents\r/F10.2_20200715-112538_scored_behaviour.csv',r'C:\Users\harry\Documents\r/F6.4_20200715-124501_scored_behaviour.csv']



In [243]:
dtypes= {'frame':int, 'behaviour':str, 'hit':str, 'values':str, 'freq':float}
rhianne_df=pd.DataFrame({})
bianca_df=pd.DataFrame({})
for i in range(len(bianca)):
    rhianne_pd = pd.read_csv(rhianne[i], dtype=dtypes)
    bianca_pd = pd.read_csv(bianca[i], dtype=dtypes)
    ##Here we cut down both arrays to be the same length
    length=np.min((len(rhianne_pd),len(bianca_pd)))
    rhianne_pd = rhianne_pd[:length]
    bianca_pd = bianca_pd[:length]
    rhianne_df = pd.concat((rhianne_pd,rhianne_df))
    bianca_df = pd.concat((bianca_pd,bianca_df))

**First we look at the total percentage of accuracy between the scorers. Note this includes significant portions of Nothing**

In [244]:
print(bianca_df['behaviour'].unique())

print(rhianne_df['behaviour'].unique())


['Nothing' 'grooming' 'rearing' 'shaking' 'digging' 'jumping_4'
 'jumping_2' 'tremors' 'shake']
['Nothing' 'tremors' 'grooming' 'rearing' 'shaking' 'digging' 'jumping_4']


In [254]:
bianca_df['behaviour'][bianca_df['behaviour']=='jumping_2'] = 'jumping'
bianca_df['behaviour'][bianca_df['behaviour']=='jumping_4'] = 'jumping'
rhianne_df['behaviour'][rhianne_df['behaviour']=='jumping_4'] = 'jumping'
bianca_df['behaviour'][bianca_df['behaviour']=='shake'] = 'Nothing'
bianca_df['behaviour'][bianca_df['behaviour']=='shaking'] = 'Nothing'

rhianne_df['behaviour'][rhianne_df['behaviour']=='shaking'] = 'Nothing'
bianca_df['behaviour'][bianca_df['behaviour']=='digging'] = 'grooming'
rhianne_df['behaviour'][rhianne_df['behaviour']=='digging'] = 'grooming'
bianca_df['behaviour'][bianca_df['behaviour']=='tremors'] = 'grooming'
rhianne_df['behaviour'][rhianne_df['behaviour']=='tremors'] = 'grooming'



bianca_df['hit'][bianca_df['hit']=='jumping_2'] = 'jumping'
bianca_df['hit'][bianca_df['hit']=='jumping_4'] = 'jumping'
rhianne_df['hit'][rhianne_df['hit']=='jumping_4'] = 'jumping'
bianca_df['hit'][bianca_df['hit']=='shake'] = 'Nothing'
bianca_df['hit'][bianca_df['hit']=='shaking'] = 'Nothing'

rhianne_df['hit'][rhianne_df['hit']=='shaking'] = 'Nothing'
bianca_df['hit'][bianca_df['hit']=='digging'] = 'grooming'
bianca_df['hit'][bianca_df['hit']=='tremors'] = 'grooming'
bianca_df['hit'][bianca_df['hit']=='tremors'] = 'grooming'
rhianne_df['hit'][rhianne_df['hit']=='tremors'] = 'grooming'
rhianne_df['hit'][rhianne_df['hit']=='digging'] = 'Nothing'

C:\Users\harry\miniconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\harry\miniconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\harry\miniconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is s

In [255]:
print("Rhianne scored a total of {} behaviours".format(np.sum((rhianne_df['freq'].values)[~np.isnan(rhianne_df['freq'].values)])))
print("Bianca scored a total of {} behaviours".format(np.sum((bianca_df['freq'].values)[~np.isnan(bianca_df['freq'].values)])))


Rhianne scored a total of 441.0 behaviours
Bianca scored a total of 628.0 behaviours


In [261]:
print(bianca_df['hit'].unique())

print(rhianne_df['hit'].unique())


[nan 'grooming' 'Nothing' 'rearing' 'jumping']
[nan 'grooming' 'Nothing' 'rearing' 'jumping']


In [262]:
length=len(bianca_df)

In [263]:
##percentage agreement overall
np.sum(rhianne_df['behaviour']==bianca_df['behaviour'])/length

0.8935036154705843

In [276]:
def get_rater_agreement(rhianne_df, bianca_df, behaviour):
    ##Here we get the parts of rhiannes data that equal Nothing
    rhianne_behaviours = rhianne_df['hit']==behaviour
    ##And the same for bianca
    bianca_behaviours = bianca_df['hit']==behaviour

    ##Now we look at the percentage of agreement between the two raters while Rhianne was not scoring a behaviour
    B_agree_R = np.sum(rhianne_df['hit'][rhianne_behaviours]==bianca_df['hit'][rhianne_behaviours])/np.sum(rhianne_behaviours)
    ##Now we look at the percentage of agreement between the two raters while Bianca was not scoring a behaviour
    R_agree_B = np.sum(bianca_df['hit'][bianca_behaviours]==rhianne_df['hit'][bianca_behaviours])/np.sum(bianca_behaviours)

    print("For {} Bianca agreed with Rhianne {} percent of the time and Rhianne agreed with Bianca {} percent of the time".format(behaviour,B_agree_R, R_agree_B))
    print("this results in an average agreement of {}".format(np.mean((B_agree_R, R_agree_B))))

In [277]:
def get_rater_agreement_within_window(rhianne_df, bianca_df, behaviour,window):
    ##Here we get the parts of rhiannes data that equal Nothing
    rhianne_behaviours = np.array(rhianne_df['hit']==behaviour)
    ##And the same for bianca
    bianca_behaviours = np.array(bianca_df['hit']==behaviour)
    rhianne_indexes=np.where(rhianne_behaviours==True)[0]
    bianca_indexes=np.where(bianca_behaviours==True)[0]
    Rcount = 0
    Bcount = 0
    rhianne_df['hit']
    rhianne_df_values = (np.array(rhianne_df['hit']))
    bianca_df_values = (np.array(bianca_df['hit']))
    window=int(window/2)
    for i in tqdm(rhianne_indexes):
        if any(rhianne_df_values[i]==bianca_df_values[i-window:i+window]):
            Rcount+=1

    print("If Rhianne scored {} there was a {} percent chance Bianca also scored that behaviour within the same {} frames\n\n".format(behaviour,Rcount/len(rhianne_indexes),window*2))

    Bcount = 0
    for i in tqdm(bianca_indexes):
        if any(bianca_df_values[i]==rhianne_df_values[i-window:i+window]):
            Bcount+=1
    print("Conversely if Bianca scored {} there was a {} percent chance Rhianne also scored that behaviour \nwithin the same {} frames".format(behaviour,Bcount/len(bianca_indexes),window*2))
    print("this results in an average agreement of {}".format(np.mean((Rcount/len(rhianne_indexes),Bcount/len(bianca_indexes)))))

In [278]:
get_rater_agreement(rhianne_df, bianca_df, "Nothing")

For Nothing Bianca agreed with Rhianne 0.009070294784580499 percent of the time and Rhianne agreed with Bianca 0.0068143100511073255 percent of the time
this results in an average agreement of 0.007942302417843912


In [279]:
for behaviour in (bianca_df['behaviour'].unique()):
    get_rater_agreement(rhianne_df, bianca_df, behaviour)


For Nothing Bianca agreed with Rhianne 0.009070294784580499 percent of the time and Rhianne agreed with Bianca 0.0068143100511073255 percent of the time
this results in an average agreement of 0.007942302417843912
For grooming Bianca agreed with Rhianne 0.009900990099009901 percent of the time and Rhianne agreed with Bianca 0.0044444444444444444 percent of the time
this results in an average agreement of 0.007172717271727172
For rearing Bianca agreed with Rhianne 0.007042253521126761 percent of the time and Rhianne agreed with Bianca 0.005681818181818182 percent of the time
this results in an average agreement of 0.006362035851472471
For jumping Bianca agreed with Rhianne 0.011764705882352941 percent of the time and Rhianne agreed with Bianca 0.011627906976744186 percent of the time
this results in an average agreement of 0.011696306429548563


In [280]:
for behaviour in (bianca_df['behaviour'].unique()):
        get_rater_agreement_within_window(rhianne_df, bianca_df, behaviour,window=180)

        

  0%|                                                                                        | 0/142 [00:00<?, ?it/s]

If Rhianne scored Nothing there was a 0.8140589569160998 percent chance Bianca also scored that behaviour within the same 180 frames


Conversely if Bianca scored Nothing there was a 0.637137989778535 percent chance Rhianne also scored that behaviour 
within the same 180 frames
this results in an average agreement of 0.7255984733473173
If Rhianne scored grooming there was a 0.49504950495049505 percent chance Bianca also scored that behaviour within the same 180 frames


Conversely if Bianca scored grooming there was a 0.24888888888888888 percent chance Rhianne also scored that behaviour 
within the same 180 frames
this results in an average agreement of 0.371969196919692


100%|███████████████████████████████████████████████████████████████████████████| 172/172 [00:00<00:00, 27006.34it/s]

If Rhianne scored rearing there was a 0.8943661971830986 percent chance Bianca also scored that behaviour within the same 180 frames


Conversely if Bianca scored rearing there was a 0.7443181818181818 percent chance Rhianne also scored that behaviour 
within the same 180 frames
this results in an average agreement of 0.8193421895006402
If Rhianne scored jumping there was a 0.9764705882352941 percent chance Bianca also scored that behaviour within the same 180 frames


Conversely if Bianca scored jumping there was a 0.9651162790697675 percent chance Rhianne also scored that behaviour 
within the same 180 frames
this results in an average agreement of 0.9707934336525308


***DONT GO BEYOND HERE***



**Next we look at the percentage agreement when scoring behaviour**


In [281]:
##Here we get the parts of rhiannes data that dont equal Nothing
rhianne_behaviours = rhianne_df['hit']!='Nothing'
##And the same for bianca
bianca_behaviours = bianca_df['hit']!='Nothing'

##Now we look at the percentage of agreement between the two raters while Rhianne was scoring a behaviour
B_agree_R = np.sum(rhianne_df['hit'][rhianne_behaviours]==bianca_df['hit'][rhianne_behaviours])/np.sum(rhianne_behaviours)
##Now we look at the percentage of agreement between the two raters while Bianca was scoring a behaviour
R_agree_B = np.sum(bianca_df['hit'][bianca_behaviours]==rhianne_df['hit'][bianca_behaviours])/np.sum(bianca_behaviours)

print("Bianca agreed with Rhianne {} percent of the time and Rhianne agreed with Bianca {} percent of the time".format(B_agree_R, R_agree_B))

Bianca agreed with Rhianne 7.648125062141016e-06 percent of the time and Rhianne agreed with Bianca 7.65026068263276e-06 percent of the time


**OK, so this looks low but Human reaction times are variable. So lets make an agreement window of half a second and consider any 
behaviours which have a inter-rater match within that window to be in agreement**

In [282]:
rhianne_indexes=np.where(rhianne_behaviours==True)[0]
bianca_indexes=np.where(bianca_behaviours==True)[0]

In [ ]:
count = 0
for i in rhianne_indexes:
    if any(rhianne_df['hit'].values[i]==bianca_df['hit'][i-22:i+22]):
        count+=1
        
print("If Rhianne scored a behaviour there was a {} percent chance Bianca also scored that behaviour within the same half second\n\n".format(count/len(rhianne_indexes)))

count = 0
for i in bianca_indexes:
    if any(bianca_df['hit'].values[i]==rhianne_df['hit'][i-22:i+22]):
        count+=1
print("Conversely if Bianca scored a behaviour there was a {} percent chance Rhianne also scored that behaviour \nwithin the same half second".format(count/len(bianca_indexes)))

**Lets now expand the window to two seconds**

In [ ]:
countR = 0
for i in rhianne_indexes:
    if any(rhianne_df['hit'].values[i]==bianca_df['hit'][i-90:i+90]):
        countR+=1
        
print("If Rhianne scored a behaviour there was a {} percent chance Bianca also scored that behaviour within two seconds\n\n".format(countR/len(rhianne_indexes)))

countB = 0
for i in bianca_indexes:
    if any(bianca_df['hit'].values[i]==rhianne_df['hit'][i-90:i+90]):
        countB+=1
print("Conversely if Bianca scored a behaviour there was a {} percent chance Rhianne also scored that behaviour \n within two seconds".format(countB/len(bianca_indexes)))

In [ ]:
print(np.mean(((countB/len(bianca_indexes)),countR/len(rhianne_indexes))))

In [274]:
countR = 0
for i in rhianne_indexes:
    if any(rhianne_df['behaviour'].values[i]==bianca_df['behaviour'][i-225:i+225]):
        countR+=1
        
print("If Rhianne scored a behaviour there was a {} percent chance Bianca also scored that behaviour within five seconds\n\n".format(countR/len(rhianne_indexes)))

countB = 0
for i in bianca_indexes:
    if any(bianca_df['behaviour'].values[i]==rhianne_df['behaviour'][i-225:i+225]):
        countB+=1
print("Conversely if Bianca scored a behaviour there was a {} percent chance Rhianne also scored that behaviour \n within five seconds".format(countB/len(bianca_indexes)))

If Rhianne scored a behaviour there was a 0.9072315164597949 percent chance Bianca also scored that behaviour within five seconds


Conversely if Bianca scored a behaviour there was a 0.6221891611263503 percent chance Rhianne also scored that behaviour 
 within five seconds


In [283]:
conservative_scorer = 'Bianca' if len(bianca_indexes)<len(rhianne_indexes) else 'Rhianne'
liberal_scorer = 'Bianca' if len(bianca_indexes)>len(rhianne_indexes) else 'Rhianne'
print("{} was the more conservative scorer, identifying behaviours on only {} frames compared to {} for {}".format(conservative_scorer, np.min((len(bianca_indexes), len(rhianne_indexes))),\
                                                                                                                   np.max((len(bianca_indexes), len(rhianne_indexes))),liberal_scorer))

Bianca was the more conservative scorer, identifying behaviours on only 522858 frames compared to 523004 for Rhianne


In [13]:
np.sum(rhianne_behaviours)/len(rhianne_df['behaviour'])



0.03589488867025189